
# 一般化

:label: `sec_generalization_basics`

最終試験に向けて熱心に準備をしている 2 人の大学生を考えてみましょう。一般に、この準備は、練習し、前年度に実施された試験を受けて能力をテストすることで構成されます。それにもかかわらず、過去の試験で良い成績を収めたからといって、重要なときに優れているという保証はありません。たとえば、エレファンティーン エリーという生徒の準備が、もっぱら過去の試験問題の答えを暗記することで構成されていたと想像してください。たとえエリーが象のような記憶力に恵まれていて、*以前に見た*質問に対する答えを完璧に思い出せたとしても、それでも、新しい (*これまで見たことのない*) 質問に直面するとフリーズする可能性があります。比較として、暗記能力は比較的劣るものの、パターンを把握するのが得意な別の生徒、帰納的アイリーンを想像してみてください。もし試験が本当に前年の再利用問題で構成されていたなら、エリーはアイリーンを優に上回る成績を収めることに注意してください。たとえアイリーンの推測パターンが 90% 正確な予測をもたらしたとしても、エリーの 100% の再現率には決して太刀打ちできません。ただし、たとえ試験がまったく新しい質問で構成されていたとしても、アイリーンは平均 90% を維持できる可能性があります。

機械学習科学者としての私たちの目標は、*パターン*を発見することです。しかし、単にデータを記憶しただけではなく、*一般的な*パターンを本当に発見したとどうやって確信できるのでしょうか?ほとんどの場合、予測はモデルがそのようなパターンを発見した場合にのみ役に立ちます。私たちが予測したいのは昨日の株価ではなく、明日の株価です。私たちは、以前に受診した患者についてはすでに診断されている疾患を認識する必要はなく、むしろ、これまで受診したことのない患者の未診断の疾患を認識する必要がある。この問題、*一般化する*パターンをどのように発見するかという問題は、機械学習、そしておそらくすべての統計の基本的な問題です。私たちはこの問題を、科学全体を巻き込むはるかに壮大な問題のほんの一部として投げかけるかもしれません: 特定の観察からより一般的なステートメントへの飛躍が正当化されるのはいつですか :cite: `popper2005logic` ?

現実では、有限のデータの集合を使用してモデルを適合させる必要があります。そのデータの一般的なスケールはドメインによって大きく異なります。多くの重要な医療問題について、私たちは数千のデータポイントしかアクセスできません。希少疾患を研究する場合、幸運にも何百もの疾患にアクセスできるかもしれません。対照的に、ラベル付き写真で構成される最大の公開データセット (例: ImageNet :cite: `Deng.Dong.Socher.ea.2009` ) には、数百万の画像が含まれています。また、Flickr YFC100M データセットなどの一部のラベルのない画像コレクションはさらに大きくなり、1 億枚を超える画像が含まれる場合もあります (cite: `thomee2016yfcc100m` )。ただし、この極端なスケールでも、利用可能なデータ ポイントの数は、1 メガピクセルの解像度で可能なすべての画像のスペースと比較すると、非常に小さいままです。有限サンプルを扱うときは常に、トレーニング データを当てはめた結果、一般化可能なパターンを発見できなかったことが判明する可能性があるというリスクを念頭に置く必要があります。

基礎となる分布よりもトレーニング データに近似する現象は、*過学習 と*呼ばれ、過学習に対処するための手法は、多くの場合、*正則化*手法と呼ばれます。統計学習理論への適切な入門に代わるものはありませんが (:citet: `Vapnik98,boucheron2005theory`参照)、始めるのに十分な直感を提供します。本書全体の多くの章で一般化について再検討し、さまざまなモデルにおける一般化の基礎となる原理について既知のことと、実際に関心のあるタスクについて改善された一般化をもたらすことが (経験的に) 判明しているヒューリスティックな手法の両方を探ります。

## トレーニングエラーと汎化エラー

標準の教師あり学習設定では、トレーニング データとテスト データが*同一の*分布から*独立して*抽出されると仮定します。これは一般に*IID 仮定*と呼ばれます。この仮定は強力ですが、そのような仮定がなければ、私たちは水の中で死んでしまうだろうということは注目に値します。分布 $P(X,Y)$ からサンプリングされたトレーニング データが、*異なる分布*$Q(X,Y)$ によって生成されたテスト データの予測方法を教えてくれるはずだとなぜ信じるべきなのでしょうか?このような飛躍を遂げるには、$P$ と $Q$ がどのように関連しているかについて強力な仮定が必要であることがわかりました。後で、分布の変化を考慮したいくつかの仮定について説明しますが、最初に $P(\cdot) = Q(\cdot)$ である IID の場合を理解する必要があります。

まず、トレーニング データセットで計算された*統計量*である*トレーニング誤差*$R_\text{emp}$ と、基礎となる分布に関して取られる*期待値*である*汎化誤差*$R$ を区別する必要があります。 。一般化誤差は、同じ基礎となるデータ分布から抽出された追加のデータ例の無限ストリームにモデルを適用した場合に発生するものと考えることができます。正式には、トレーニング誤差は*合計*として表されます ( :numref: `sec_linear_regression`で同じ表記を使用します)。

 $$R_\text{emp}[\mathbf{X}, \mathbf{y}, f] = \frac{1}{n} \sum_{i=1}^nl(\mathbf{x}^{( i)}, y^{(i)}, f(\mathbf{x}^{(i)})),$$

一方、一般化誤差は積分として表されます。

 $$R[p, f] = E_{(\mathbf{x}, y) \sim P} [l(\mathbf{x}, y, f(\mathbf{x}))] = \int \int l(\mathbf{x}, y, f(\mathbf{x})) p(\mathbf{x}, y) ;d\mathbf{x} dy.$$

問題として、汎化誤差 $R$ を正確に計算することは決してできません。密度関数 $p(\mathbf{x}, y)$ の正確な形式を教えてくれる人は誰もいません。さらに、データ ポイントの無限のストリームをサンプリングすることはできません。したがって、実際には、私たちのモデルを、私たちのモデルから差し控えられたサンプル$\mathbf{X}&#39;$とラベル$\mathbf{y}&#39;$のランダムな選択で構成される独立したテストセットに適用することによって、汎化誤差*を推定する*必要があります。トレーニングセット。これは、経験的トレーニング誤差の計算と同じ式をテスト セット $\mathbf{X}&#39;、\mathbf{y}&#39;$ に適用することで構成されます。

重要なのは、テスト セットで分類器を評価するとき、*固定の*分類器 (テスト セットのサンプルに依存しない) を使用しているため、その誤差の推定は単なる平均推定の問題です。ただし、トレーニングセットについては同じことが言えません。最終的に得られるモデルはトレーニング セットの選択に明示的に依存するため、トレーニング誤差は一般に基礎となる母集団の真の誤差の偏った推定値になることに注意してください。一般化の中心的な問題は、トレーニング誤差が母集団誤差 (したがって汎化誤差) に近づくといつ予測すべきかということです。

### モデルの複雑さ

古典理論では、単純なモデルと豊富なデータがある場合、トレーニング誤差と汎化誤差が近くなる傾向があります。ただし、より複雑なモデルや少数の例を扱う場合、トレーニング誤差は減少しますが、汎化ギャップは拡大すると予想されます。これは驚くべきことではありません。 $n$ サンプルのあらゆるデータセットに対して、たとえランダムに割り当てられたとしても、任意のラベルに完全に適合するパラメーターのセットを見つけることができるほど表現力豊かなモデル クラスを想像してください。この場合、たとえトレーニング データを完全に適合させたとしても、汎化誤差についてどのように結論付けることができるでしょうか?私たちが知っている限りでは、私たちの一般化エラーはランダムな推測にすぎない可能性があります。

一般に、モデル クラスに制限がない場合、トレーニング データのフィッティングのみに基づいて、モデルが一般化可能なパターンを発見したと結論付けることはできません :cite: `vapnik1994measuring` 。一方、モデル クラスが任意のラベルを当てはめることができなかった場合、モデル クラスはパターンを発見したはずです。モデルの複雑さに関する学習理論のアイデアは、反証可能性の基準を形式化した影響力のある科学哲学者であるカール ポパーのアイデアからインスピレーションを得ました。ポパーによれば、あらゆる観察を説明できる理論はまったく科学理論ではありません。結局のところ、それが何らかの可能性を排除していないとしたら、それは世界について私たちに何を教えてくれたのでしょうか？要するに、私たちが望んでいるのは、私たちが行うと考えられる観察を説明*できない*にもかかわらず、*実際に*私たちが行っている観察とたまたま一致する仮説です。

さて、モデルの複雑さの適切な概念を正確に構成するものは複雑な問題です。多くの場合、より多くのパラメータを持つモデルは、より多くの任意に割り当てられたラベルを適合させることができます。ただし、これは必ずしも真実ではありません。たとえば、カーネル メソッドは無限の数のパラメータを持つ空間で動作しますが、その複雑さは他の手段によって制御されます (cite: `scholkopf2002learning` )。多くの場合役立つ複雑さの概念の 1 つは、パラメーターが取り得る値の範囲です。ここで、パラメーターが任意の値を取ることが許可されているモデルは、より複雑になります。次のセクションで、最初の実用的な正則化テクニックである*Weight Decay*を紹介するときに、このアイデアを再検討します。特に、実質的に異なるモデル クラス (たとえば、決定木とニューラル ネットワーク) のメンバー間で複雑さを比較するのは困難な場合があります。

この時点で、ディープ ニューラル ネットワークを導入するときに再検討するもう 1 つの重要な点を強調する必要があります。モデルが任意のラベルを当てはめることができる場合、トレーニング誤差が低いことが必ずしも汎化誤差が低いことを意味するわけではありません。*ただし、必ずしも汎化誤差が高いことを意味するわけではありません。*私たちが自信を持って言えるのは、トレーニング誤差が低いというだけでは汎化誤差が低いことを証明するには十分ではないということだけです。ディープ ニューラル ネットワークは、まさにそのようなモデルであることが判明しました。実際には十分に一般化できますが、トレーニング エラーのみに基づいて多くのことを結論付けるには強力すぎるためです。このような場合、事後的に一般化を証明するには、ホールドアウト データにさらに依存する必要があります。ホールドアウト データ、つまり検証セットのエラーは*検証エラー*と呼ばれます。

## 過小適合か過適合か?

トレーニング エラーと検証エラーを比較するときは、2 つの一般的な状況に留意する必要があります。まず、トレーニング エラーと検証エラーがどちらも大きいにもかかわらず、それらの間にわずかなギャップがある場合に注意したいと思います。モデルがトレーニング エラーを削減できない場合は、モデルが単純すぎる (つまり、表現力が不十分) ため、モデル化しようとしているパターンを捉えることができないことを意味している可能性があります。さらに、トレーニング エラーと汎化エラーの間の*汎化ギャップ*($R_\text{emp} - R$) が小さいため、より複雑なモデルを使用しても問題を解決できると信じる理由があります。この現象は*アンダーフィッティング*として知られています。

一方で、上で説明したように、トレーニング誤差が検証誤差よりも大幅に低い場合、つまり重度の*過学習*を示す場合に注意する必要があります。過剰適合は必ずしも悪いことではないことに注意してください。特に深層学習では、最良の予測モデルは、多くの場合、ホールドアウト データよりもトレーニング データに対してはるかに優れたパフォーマンスを発揮します。結局のところ、私たちは通常、汎化誤差を低くすることを重視しており、そのギャップが障害になる場合にのみギャップを気にします。トレーニング誤差がゼロの場合、汎化ギャップは汎化誤差と正確に等しく、ギャップを減らすことによってのみ進歩できることに注意してください。

### 多項式曲線フィッティング

:label: `subsec_polynomial-curve-fitting`

過学習とモデルの複雑さに関する古典的な直観を説明するために、次のことを考えてみましょう。単一の特徴 $x$ と対応する実数値ラベル $y$ で構成されるトレーニング データが与えられた場合、次数 $d$ の多項式を見つけようとします。

 $$\hat{y}= \sum_{i=0}^dx^i w_i$$

ラベル $y$ を推定します。これは単なる線形回帰問題であり、すべての $ に対して $x^0 = 1$ であるため、特徴は $x$ の累乗で与えられ、モデルの重みは $w_i$ で与えられ、バイアスは $w_0$ で与えられます。 x$。これは単なる線形回帰問題であるため、損失関数として二乗誤差を使用できます。

高次多項式関数は、より多くのパラメーターを持ち、モデル関数の選択範囲が広いため、高次多項式関数は低次多項式関数よりも複雑になります。トレーニング データセットを修正すると、高次の多項式関数は、低次の多項式と比較して常に低い (最悪の場合でも等しい) トレーニング誤差を達成する必要があります。実際、各データ例が $x$ という個別の値を持つ場合は常に、データ例の数に等しい次数を持つ多項式関数がトレーニング セットに完全に適合します。多項式の次数 (モデルの複雑さ) と過適合と過適合の関係を :numref: `fig_capacity_vs_error`で視覚化します。 

![](../img/capacity-vs-error.svg) :label: `fig_capacity_vs_error`

### データセットのサイズ

上記の限界がすでに示しているように、留意すべきもう 1 つの大きな考慮事項はデータセットのサイズです。モデルを修正すると、トレーニング データセット内のサンプルが少なくなるほど、過学習が発生する可能性が高くなります (そして深刻になります)。トレーニング データの量が増えると、一般化誤差は通常減少します。さらに、一般に、データが増えても問題はありません。固定タスクとデータ分散の場合、モデルの複雑さがデータ量よりも急激に増加してはなりません。より多くのデータが与えられると、より複雑なモデルの適合を試みる可能性があります。十分なデータがなければ、単純なモデルを打ち負かすのはより困難になる可能性があります。多くのタスクでは、数千のトレーニング サンプルが利用可能な場合にのみ、ディープ ラーニングが線形モデルよりも優れたパフォーマンスを発揮します。ディープラーニングの現在の成功は、インターネット企業、安価なストレージ、接続されたデバイス、および経済の広範なデジタル化から得られる豊富な大規模なデータセットに大きく貢献しています。

## モデルの選択

:label: `subsec_generalization-model-selection`

通常、さまざまな方法 (さまざまなアーキテクチャ、トレーニング目標、選択された機能、データの前処理、学習率など) が異なる複数のモデルを評価した後でのみ、最終的なモデルを選択します。多くのモデルの中から選択することを、適切には*モデル選択*と呼びます。

原則として、すべてのハイパーパラメータを選択するまではテスト セットに触れるべきではありません。モデル選択プロセスでテスト データを使用すると、テスト データが過剰適合してしまうリスクがあります。そうなると、私たちは深刻な問題に直面することになるでしょう。トレーニング データをオーバーフィットした場合は、正直さを保つために常にテスト データの評価が行われます。しかし、テストデータを過剰適合させた場合、どうやってそれを知ることができるでしょうか?複雑さを厳密に制御できるモデルであっても、これが不合理な結果につながる可能性がある例については、:citet: `ong2005learning`を参照してください。

したがって、モデルの選択においてテストデータに依存すべきではありません。それでも、モデルの選択にトレーニング データのみに依存することはできません。モデルのトレーニングに使用するデータ自体の汎化誤差を推定することができないためです。

実際のアプリケーションでは、状況はさらに曖昧になります。理想的には、最も優れたモデルを評価するため、または少数のモデルを相互に比較するために、テスト データに一度だけ触れることが望ましいですが、実際のテスト データは 1 回使用しただけで破棄されることはほとんどありません。実験のたびに新しいテストセットを用意する余裕はほとんどありません。実際、何十年にもわたってベンチマーク データをリサイクルすると、[画像分類](https://paperswithcode.com/sota/image-classification-on-imagenet)や[光学式文字認識](https://paperswithcode.com/sota/image-classification-on-mnist)などのアルゴリズムの開発に大きな影響を与える可能性があります。

*テスト セットでのトレーニング*の問題に対処する一般的な方法は、データを 3 つの方法で分割し、トレーニング データセットとテスト データセットに加えて*検証セットを*組み込むことです。その結果、検証データとテストデータの間の境界が憂慮すべきほどあいまいな、あいまいな実践が行われます。特に明記されていない限り、本書の実験では、真のテスト セットを使用せずに、正しくトレーニング データと検証データと呼ぶべきものを実際に使用しています。したがって、本書の各実験で報告されている精度は実際には検証の精度であり、真のテスト セットの精度ではありません。

### 相互検証

トレーニング データが不足している場合、適切な検証セットを構成するのに十分なデータを保持する余裕さえない可能性があります。この問題に対する一般的な解決策の 1 つは、 $K$*分割相互検証を*採用することです。ここでは、元のトレーニング データが $K$ の重複しないサブセットに分割されます。次に、モデルのトレーニングと検証が $K$ 回実行され、毎回 $K-1$ サブセットでトレーニングされ、別のサブセット (そのラウンドのトレーニングに使用されなかったサブセット) で検証されます。最後に、$K$ 実験の結果を平均することによって、トレーニングと検証の誤差が推定されます。

## まとめ

このセクションでは、機械学習における一般化の基礎のいくつかを検討しました。これらのアイデアの一部は、より深いモデルに到達すると複雑で直感に反したものになります。そこでは、モデルはデータをひどく過剰適合させる可能性があり、複雑さの関連概念は暗黙的かつ直感に反したものになる可能性があります (たとえば、より多くのパラメーターを備えた大規模なアーキテクチャの方がより一般化が容易です)。いくつかの経験則を残しておきます。
1. モデルの選択には検証セット (または $K$ *-fold Cross-validation* ) を使用します。
1. より複雑なモデルでは、多くの場合、より多くのデータが必要になります。
1. 複雑さの関連概念には、パラメーターの数と、パラメーターが取り得る値の範囲の両方が含まれます。
1. 他のすべてを同じにすると、ほとんどの場合、より多くのデータがより良い一般化につながります。
1. この一般化に関する話はすべて IID の仮定に基づいています。この仮定を緩和して、トレーニング期間とテスト期間の間で分布がシフトすることを許容すると、さらなる (おそらくより穏やかな) 仮定がなければ一般化について何も言えなくなります。

## 演習
1. 多項式回帰の問題を正確に解けるのはいつですか?
1. 依存する確率変数によって問題を IID データとして扱うことが推奨できない例を少なくとも 5 つ挙げてください。
1. トレーニングエラーがゼロになることを期待できるでしょうか?どのような状況で汎化誤差がゼロになるでしょうか?
1.  $K$ 倍の相互検証の計算コストが非常に高いのはなぜですか?
1.  $K$ 倍の交差検証誤差推定値に偏りがあるのはなぜですか?
1.  VC 次元は、関数のクラスの関数によって任意のラベル ${\pm 1}$ で分類できる点の最大数として定義されます。関数のクラスがどれほど複雑かを測定するのにこれが良い考えではないのはなぜでしょうか?ヒント: 関数の大きさはどうでしょうか?
1. マネージャーは、現在のアルゴリズムがあまりうまく機能しない難しいデータセットをあなたに提供します。もっとデータが必要であることを彼にどうやって正当化しますか?ヒント: データを増やすことはできませんが、減らすことはできます。



[ディスカッション](https://discuss.d2l.ai/t/97)
